# 추천 시스템 심화

## 기존 추천시스템 정리
1. 내용 기반 필터링
    * 콘텐츠의 유사도를 기반으로 사용자에게 추천하는 방법
    * 사용자가 이용하거나 선택한 아이템과 유사한 아이템을 추천하는 방삭
2. 협업 필터링
    * 서비스 내에 있는 다수의 사용자로부터 얻은 선호도 정보를 기반으로 사용자에게 추천하는 방법
        * cf. 콘텐츠 기반 필터링: 한 사용자의 행동 이력을 기준으로 추천
3. 하이브리드 필터링
    * 2가지 이상의 다양한 종류의 추천시스템 알고리즘을 조합하여 만드는 알고리즘

## 협업 필터링의 구현 방식
1. 기업 기반 : 실제 평점 데이터를 메모리에 저장해두고, 유사도를 계산하는 방식
    1. 사용자 기반 협업 필터링 
    * 작동 원리
        * 나와 다른 사용자 간의 유사도를 계산
        * 유사도가 높은 상위 k명의 이웃을 선정
        * 이웃들이 선호했지만 내가 보지 못한 아이템을 추천
    * 문제점: 사용자가 많아질 수록 계싼량이 폭발적으로 늘어나고, 사용자 취향이 매우 다양하여 유사도를 정확히 찾기 어려움
    2. 아이템 기반 협업 필터링
    * 아이디어: 함께 선호되는 아이템을 찾습니다
    * 작동 원리
        * 모든 아이템 쌍 간의 유사도를 계산
        * 내가 선호한 아이템과 유사한 아이템을 찾아서 추천
    * 장점: 아이템은 사용자보다 변화가 적어 유사도 행렬이 비교적 안정적 

2. 기억 기반 협업 필터링의 한계
    1. 데이터 희소성 문제
    * 사용자가 전체 아이템 중 극히 일부에만 평점을 매김
    * 행렬이 비어있어서, 유사도를 계산할 만한 공통의 아이템이 부족
    2. 확장성 문제
    * 사용자와 아이템 수가 증가할수록 유사도를 계산해야하는 쌍의 수가 기하급수적으로 늘어남
    * 대규모 서비스에서는 실시간으로 유사도를 계산하는 것이 거의 불가능
    3. 새로운 아이템 문제
    * 새로운 아이템은 상호작용 이력이 없음 -> 그래서 추천될 기회가 없다는 단점

3. 모델 기반 협업 필터링의 등장
    * 위의 한계를 극복하고자... -> 관측된 평점 데이터로부터 유저/아이템의 숨겨진 취향/특성을 학습 -> 잠재 요인을 이용하여 비어있는 평점을 예측
    * 유사도 계산이 아니라 내적 연산을 통해 평점 예측


## 1. 인트로 - 모델 기반
### 1-1. 잠재 요인과 행렬 분해
1. 잠재요인 : 숨겨진 차원
* 사람이 명시적으로 이해x -> 모델이 스스로 학습한 추상적인 개념
* 우리는 벡터 값을 보고 명시적으로 이름을 붙일 수는 없지만, 간접적으로는 파악할 수 있다.
* 평점 예측(내적): 두 잠재요인 벡터의 내적으로 예측됨
핵심:**두 잠재요안 벡터의 유사도(내적)가 평점을 결정**

#### 1.1.2 행렬 분해
* 가설: 평점 행렬이 사용자의 취향과 아이템의 특성 벡터의 내적으로 설명될 수 있다.
* 목표: **가장 적절한 잠재 요인**
* 평점 예측: 잠재요인 개수(차원)

#### 1.1.3 MF 수행 방식
* 행렬 분해 방식에는 크게 4종류가 있습니다
    * SVD : 수학적 분해 기법 | 행령 대수학에 기반하여 행렬 분해
    * SGD : 최적화 알고리즘 | 경사 하강법 기반의 알고리즘 -> 관측된 평점에 대해서만 학습 진행
    * NMF : 제약 조건 기반 모델 | 분해된 행렬에 비음수라는 제약 조건을 추가하여 잠재 요인의 해석력을 높임
    * PMF : 확률적 모델 | 평점 예측에 확률 분포를 사용하여 불확실성까지 모델링하는 통계적 접근법
* 특이값 분해
    * SVD란, 어떤 직사각형 행렬이든 다음의 세 행렬로 분해할 수 있는 방법
    * 또한 모든 특잇갓(대각행렬)과 벡터를 사용하는 Full SVD와 Truncated SVD가 있습니다. 
    * SVD를 이용한 MF 구현
        1. 결측치 처리
        2. SVD 수행
        3. 차원 축소
        4. 잠재 요인 추출
* Stochastic Gradient Desent
: 수학적으로 우아하지만, 실제 추천 시스템의 희소한 평점 행렬에 적용하기에는 한계가 있음
    * 한계
    1. 데이터 희소성 처리 불가: SVD는 행렬이 비어있지 않은 밀집 행렬이
    2. 계산 복잡성: 대규모 행렬에 대해 SVD를 계산하는 것은 매우 큰 비용, 실시간 업데이트가 어려움
    * SGD의 목표: 목적 함수
        * 오차의 제곱합을 최소호하고, 과적합을 방지하기 위한 정규화 항을 추가
        * SGD를 이용한 MF 구현
            * 초기화
            * 반복 학습
            * 오차 계산
            * 잠재 벡터 업데이트
* NMF: 모든 원소가 0 이상이어야 한다는 제약 조건을 추가한 행렬 분해 기법
* 대부분의 현실 데이터는 음수일 수 없는 점을 이용 -> 직관적 의미 부여 가능
* 장점: 직관적 의미 가능
    * 잠재 요인 값이 음수가 될 수 없기 때문에, 잠재요인 값이 유저의 선호도나 아이템의 특성 강도를 나타낸다고 직관적으로 해석할 수 있음.
* PMF 
    * 누가 무엇을 좋아할지 예측하는데에 목표
    * PMF는 확률적 모델로 접근
    * 행렬 분해가 불확실성을 포함

## 2. 최근 추천시스템의 방향성
### 2-1. VAE 기반 협업 필터링
* 사용자의 아이템 상호작용 벡터를 입력으로 받아, 잠재 공간에서 재구성하도록 VAE를 학습하는 방식
* 구조
    * 인코더: 사용자 이력을 잠재변수 분포로 매핑
    * 디코더: z로부터 아이템 분포를 복원
* 장점
    * 기존 MF처럼 단일 포인트 벡터가 아니라 확률적 잠재 분포로 사용자 선호를 모델링하여 불확실성 반영
    * 희소한 상호작용 데이터에서도 강건
* 예시 모델: Mult-VAE

### 2.1.1 MF의 한계와 딥러닝의 등장
: 행렬 분해(MF)가 희소성과 확장성 문제
* MF의 근본적 한계
    * 선형성의 한계: 평점 예측이 단순한 잠재 벡터의 내적으로 이루어짐 | 
    * 불확실성 무시: 사용자 취향을 단 하나의 벡터값으로만 표현 |
    * 한계를 극복하기 위해 등장한 방식 중 하나가 VAE

#### 2.1.2 왜 AE가 아닌 VAE인가?
딥러닝의 비선형성을 추천시스템에 도입한 초기 시도: 일반적인 AE
* -> VAE는 불확실성을 모델링하며 추천시스템의 고질적인 문제인 희소성에 대한 해답을 제시
* AE는 데이터가 부족하면 단일 추정치가 불안정하고 틀리기 쉬워서, 노이즈에 취약
* VAE는 취향의 불확실성(데이터 부족으로 인한 오차범위)까지 모델링에 포함해 안정성을 높임

#### 2.1.3. VAE 기반 CF의 작동구조
* 사용자의 아이템 상호작용 벡터를 입력으로 받아, 잠재공간에서 재구성하도록 학습
| 구성 요소 | 역할 |
| --- | --- |
| **입력** ($x$) | 사용자의 아이템 상호작용 벡터 |
| **인코더** | 사용자 이력을 잠재변수 $z$ 분포로 매핑 |
| **잠재 벡터** ($z$) | 샘플링된 최종 취향 벡터 |
| **디코더** | $z$로부터 아이템 분포를 복원 |

#### 2.1.4. VAE의 비밀 병기: 목적함수와 KL Divergence
: 우수성은 목적함수에 있으며, 아래와 같이 두 가지 목표를 동시에 최적화
* 재구성 오차 최소화(첫번째항): 디코더가 잠재벡터를 사용해 원래 입력을 비슷하게 복원하게 만듦
* KL Divergence 항 (두번째항): 인코더가 출력하는 잠재분포가 사전 분포와 유사하도록 강제 -> 이 항으로 잠재공간을 넓고 부드럽게 유지하도록 함

* 역할: 희소성 강건성의 비밀
    * AE 문제: 불규칙한 고안에 잠재 벡터에 있으면 보간하여 예측하기 어렵다
    * VAE의 강점: 잠재공간을 정규 분포 근처로 압축해 부드럽게 만듦
    * 이를 통해 데이터가 부족한 유저라도 주번에 풍부한 데이터로부터 정보를 자연스럽게 보간 받아 안정적인 예측 할 수 있게 함
* VAE - CF의 장점
    * 확률적 잠재 분포 모델링(불확실성 반영)
    * 희소한 상호작용 데이터에서도 강건
    * 비선형성 학습
        * 위에 세가지를 통해 희소데이터에서도 강건한 추천이 가능해짐

### 2.2 GNN 기반 사용자 선호 예측 
0. 필요성: 관계가 순서라는 풍부한 정보를 활용하지 못함!
1. GNN : 누가 뭐를 좋아했다는 집합을 넘어 서로 연결된 거대한 관계망을 형성한다! -> 본질적으로 그래프!!
    * 그래프: 노드와 노드를 잇는 선(엣지)를 모아 구성한 자료 구조
        * 노드 : 현실 세계의 개체
        * 엣지 : 개체간 관계나 상호작용
    * 사용 이유: 추상적인 개념을 다루기에 적합 | 격자 구조가 아야도 표현하고 학습할 수 있음!


2. GNN의 구조
    1. 그래프 구조 생성
    2. 메시지 패싱
        1. 메시지 생성
        2. 집계
        3. 갱신
    3. 최종 출력
* 핵심 아이디어: 내 이웃노드의 정보를 나에게 전달/요약

1. 그래프 구조 생성 
* 노드와 엣지를 정의한 후 각각의 피처를 담아 표현
2. 메시지 패싱 : 노드 간 정보를 전달하고 상호작용 하는 방법으로, 메시지 생성-집계-갱신의 세 단계로 이루어짐
    1. 타겟 노드 선정
    2. 메시지 생성 : 메시지는 이웃에게 보내는 자신의 정보(임베딩)
    3. 메시지 집계: 이웃 노드에게 받은 메시지를 하나의 벡터로 통합
    4. 갱신 : 이웃 정보와 자신의 이전 상태를 결합해 새로운 임베딩을 계싼
* 각 노드는 이웃 노드에만 메시지를 전달할 수 있어서 정해진 순서에 따라 다음 노드에 전달

#### 2.2.3 LIght CNN
* 주요 GNN 아키텍처
    * GCN (Graph Convolutional Network) : 그래프 위에서 Convolution 연산 수행. 이웃 노드와 타겟 노드에 동일한 파라미터 적용함.
    * GRN (Graph Recurrent Network) : RNN + GNN 결합. 시간에 따라 노드 특성이 업데이트 되어 그래프의 동적 변화 포착
    * GAT (Graph Attention Network) : 이웃 노드 간 중요도를 학습해 가중 평균. 타겟 노드의 이웃 노드 중 중요한 노드에 가중치를 둠.
    * GraphSAGE : 대규모 그래프에 대해 샘플링 기반으로 학습. 이웃 노드와 타겟 노드의 가중치 값을 더하지 않고, concat함.
* LIght CNN은 추천 시스템에 특화된 GNN으로 복잡성을 제거하고 최적화한 ㅂ버정
    * GNN 구성 요소를 2가지로 구성
        * 특징 변환 제거
        * 비선형 활성화 함수 제거
    * => 오직 정규화된 이웃 임베딩의 합만으로 구성

#### 2.2.4. GNN의 활용사례
* 소셀네트워크, 추천시스템, 화학 및 생명과학, 금융 및 보안, 프로그램 분석

#### 2.2.5 GNN의 장단점
* 장점 
1. 고차원 협업 시그널 포착
2. 정형화되지 않은 데이터 학습 가능 : 관계를 학습하기 때문
3. 콜드 스타드 문제 완화에 효과적
* 단점
    1. 과평탄화 문제
    2. 과압축 문제
    3. 계산 복잡도 높아짐
    4. 해석의 어려움

#### 2.3. 트랜스포머 기반 추천 시스템
1. 기존 추천 시스템의 순차적 한계
    1. CF의 순차적 한계
        * 순서 무시: 순서 바뀌어도 동일하게 간주
        * 문맥 부족
        * 예측 한계: 일반적인 선호도 찾는데에 초점 맞춰져 있음
2. 순차적 한계
    * 지역적 정보 전파의 한계: 보통 연결된 이웃 관계만 보기 때문에 정보가 희석됨
    * 장기 의존성 문제

3. 트랜스포머: 사용자 행동을 순서로 접근
* 행동 순서가 중요한 이유: 의도의 파악
4. 트랜스포머의 핵심: 셀프 어텐션
    * 모든 아이템 간의 관계를 거리에 상관없이 동시 계산함
    * 연산과정 : 관련성을 계산해 가중치를 만듦
        1. **Attention Score 계산 (Q·K):** Query(현재 토너)가 모든 Key(프린터, 가습기, ...)와 얼마나 유사한지(관련 있는지) 유사도(주로 내적)를 통해 '관련성 점수'를 계산합니다.
        2. **가중합 (Weighted Sum) (Score·V):** 이 '관련성 점수'를 (Softmax를 거쳐) '가중치'로 변환합니다.
        3. 이 가중치를 각 아이템의 Value(실제 내용물)에 곱하여 모두 더합니다. (가중 평균)

5. 트랜스포머 기반 주요 추천 시스템 모델
    1. SASRec
    * 단방향 어텐샨
    * 다음 아이템 예측하는 것이 목표
    * 특징: 과거 정보만 활용하여 시간의 흐름을 명확하게 반영
    * 문제점: 계산량 많음, 단반향성 -> 이 두가지 문제점을 보완해서 등장한 알고리즘이 - BERT4rec
    2. BERT4rec
    * 양방향 어텐션
    * 마스크 토큰으로 일부러 가르고, 가려진 아이템이 과거/미래 모든 아이템을 찾모
    * 과거와 미래를 모두 고려하고 학습 과정에서 시퀀스 전체의 선호도를 파악하는 데 더 초점

### LLM의 시대
* 관점의 전환: 추천을 랭킹 문제가 아닌 언어 문제로 푼다
    * LLM은 추천을 점수 계산이 아닌 문장으로 출력
    => 단순한 랭킹을 보여주는 것이 아니라, 사용자의 상황에 맞게 이유를 덧붙인 서술형 추천까지 가능
* 새로운 가능성
    * 대화형 추천: 자연어 요청을 직접 이해하고 추천
    * Zero-shot 추천 : 학습 데이터에 없던 새로운 아이템도 아이템의 ‘텍스트 설명’만으로 그럴싸한 추천 가능
* P5: LLM이 풀 수 있도록 '하나의 거대한 텍스트 프롬프트' 문제로 통일시킨 모델
* TALLRec : P5처럼 처음부터 다 학습시키긴 비싸니까, 이미 잘 만들어진 범용 LLM(e.g., LLaMA)을 추천'작업에 맞게 효율적으로 튜닝(Tuning)하는 프레임워크
=> 단순한 성능 향상이 아니라 상호작용하는 방식 자체를 근본적으로 바꾸고 있음

### AI
* 모델 성능이 다가 아니다.
1. 인기편향: 추천 시스템이 소수의 인기 아이템을 과도하게 노출하는 반면, 다수의 비인기 아이템이 받아야 할 관심을 받지 못하는 현상
2. 설명가능성(XAI)
* 이걸 왜 추천했는지 설명할 수 있는가?
* 왜 추천했는지 설득해야 하고 추천의 근거를 제시해야함 ! 즉, 책임감 있는 모델
* 딥러닝 모델이 좋아보이겠지만 비용문제로 딥러닝을 전면 도입하기에 너무 비싸기 때문에 여전히 현업에서는 선형 기반 추천시스템과 병행되고 있음
    * 1단계: 후보군 생성
    * 2단계; 순위 매기기